In [1]:
colors = ['white','green','yellow','red','brown','pink']

In [2]:
color_rdd = sc.parallelize(colors)

In [3]:
keyval_rdd = color_rdd.map(lambda x:(x,len(x)))

In [4]:
color_df = keyval_rdd.toDF(['color','length']) 

In [5]:
color_df

DataFrame[color: string, length: bigint]

In [6]:
color_rdd.getNumPartitions()

2

In [7]:
keyval_rdd.getNumPartitions()

2

In [8]:
color_df.rdd.getNumPartitions()

2

In [9]:
type(color_df.rdd)

pyspark.rdd.RDD

In [10]:
type(color_df)

pyspark.sql.dataframe.DataFrame

In [11]:
 color_df.printSchema()

root
 |-- color: string (nullable = true)
 |-- length: long (nullable = true)



In [12]:
color_df.count() 


6

In [13]:
color_df.show(2)


+-----+------+
|color|length|
+-----+------+
|white|     5|
|green|     5|
+-----+------+
only showing top 2 rows



In [14]:
color_df.columns


['color', 'length']

In [15]:
color_df.drop('length').show() 


+------+
| color|
+------+
| white|
| green|
|yellow|
|   red|
| brown|
|  pink|
+------+



# ** สังเกตว่า color_df ยังคงมีค่าเดิม

In [16]:
color_df.show()

+------+------+
| color|length|
+------+------+
| white|     5|
| green|     5|
|yellow|     6|
|   red|     3|
| brown|     5|
|  pink|     4|
+------+------+



# ** หากเราต้องการเก็บค่าไว้ ให้ทำแบบนี้

In [17]:
new_color_df = color_df.drop('length')

In [18]:
new_color_df.show()

+------+
| color|
+------+
| white|
| green|
|yellow|
|   red|
| brown|
|  pink|
+------+



In [19]:
color_df.filter(color_df.length.between(4,5))\
.select(color_df.color.alias("mid_length")).show()


+----------+
|mid_length|
+----------+
|     white|
|     green|
|     brown|
|      pink|
+----------+



In [20]:
color_df.filter(color_df.length > 4) .filter(color_df[0]!="white").show()

+------+------+
| color|length|
+------+------+
| green|     5|
|yellow|     6|
| brown|     5|
+------+------+



In [21]:
color_df.filter((color_df.length > 4)&(color_df[0]!="white")).show()

+------+------+
| color|length|
+------+------+
| green|     5|
|yellow|     6|
| brown|     5|
+------+------+



# ** โค้ดบรรทัดที่ 21 อาจเขียนพลาดได้กรณี reuse บรรทัดนี้อีกแล้วลืมเปลี่ยนชื่อตัวแปร ...ต่อไปนี้คือ วิธีการลดข้อผิดพลาด

In [22]:
from pyspark.sql import functions as sparkF

In [23]:
color_df.filter((sparkF.col('length') >4)&(sparkF.col('color')!='white')).show()

+------+------+
| color|length|
+------+------+
| green|     5|
|yellow|     6|
| brown|     5|
+------+------+



In [24]:
color_df.filter(color_df['length']>=4).sort("length", 'color',ascending=False).show()

+------+------+
| color|length|
+------+------+
|yellow|     6|
| white|     5|
| green|     5|
| brown|     5|
|  pink|     4|
+------+------+



In [25]:
color_df.orderBy('length','color').take(4)

[Row(color='red', length=3),
 Row(color='pink', length=4),
 Row(color='brown', length=5),
 Row(color='green', length=5)]

In [26]:
from pyspark.sql import Row
spark.createDataFrame([Row(color='red', length=3),
 Row(color='pink', length=4),
 Row(color='brown', length=5),
 Row(color='green', length=5)]).show()

+-----+------+
|color|length|
+-----+------+
|  red|     3|
| pink|     4|
|brown|     5|
|green|     5|
+-----+------+



In [27]:
color_df.sort(color_df.length.desc(),\
color_df.color.asc()).show()


+------+------+
| color|length|
+------+------+
|yellow|     6|
| brown|     5|
| green|     5|
| white|     5|
|  pink|     4|
|   red|     3|
+------+------+



In [28]:
color_df.groupBy('length').count().show()

+------+-----+
|length|count|
+------+-----+
|     6|    1|
|     5|    3|
|     3|    1|
|     4|    1|
+------+-----+



In [29]:
color_df.describe().show()

+-------+------+------------------+
|summary| color|            length|
+-------+------+------------------+
|  count|     6|                 6|
|   mean|  null| 4.666666666666667|
| stddev|  null|1.0327955589886444|
|    min| brown|                 3|
|    max|yellow|                 6|
+-------+------+------------------+

